In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [8]:
class Char_CNN(nn.Module):
    
    def __init__(self, conv_layers = [
                    [256, 7, 3],
                    [256, 7, 3],
                    [256, 3, None],
                    [256, 3, None],
                    [256, 3, None],
                    [256, 3, 3]
                    ],
                 fully_layers = [1024, 1024],
                 l0 = 1014,
                 alphabet_size = 69,
                 nb_classes = 4,
                 batch_size = 32,
                 th = 1e-6):
        
        super(Char_CNN,self).__init__()
        self.conv_layers = conv_layers
        self.fully_layers = fully_layers
        self.batch_size = batch_size
        self.nb_classes = nb_classes
        
        self.convs = []
        self.linear = []
        self.max_pool = nn.MaxPool1d(3)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout()
        
        in_feat = alphabet_size
        for out_feat, kernel_size, max_pool in conv_layers:
            conv = nn.Conv1d(in_feat, out_feat, kernel_size)
            self.convs.append(conv)
            in_feat = out_feat
        
        l6 = int((l0 - 96)/27)
        in_feat = l6*out_feat
        
        for out_feat in fully_layers:
            self.linear.append(nn.Linear(in_feat, out_feat))
            in_feat = out_feat
        
        self.classifier = nn.Linear(in_feat, nb_classes)
        
        if self.nb_classes == 2:
            self.class_non_lin = nn.Sigmoid()
        else:
            self.class_non_lin = nn.Softmax()
        

    def forward(self, x):
        out = x 
        print(out.size())
        for conv in self.convs[:2]:
            out = conv(out)
            out = self.relu(out)
            out = self.max_pool(out)
            print(out.size())
            
        for conv in self.convs[2:5]:
            out = conv(out)
            out = self.relu(out)
            print(out.size())
            
            
        out = self.convs[5](out)
        out = self.relu(out)
        out = self.max_pool(out)
        print(out.size())
        
        out = out.view(batch_size, -1)
        print(out.size())
        
        for lin in self.linear:
            out = lin(out)
            out = self.relu(out)
            out = self.dropout(out)
            print(out.size())

            
        out = self.classifier(out)
        out = self.class_non_lin(out)
        
        return out
    
    def init(self):
        for conv in self.convs:
            nn.init.normal(conv.weight, mean=0, std=0.02)

In [9]:
conv_layers = [
                    [256, 7, 3],
                    [256, 7, 3],
                    [256, 3, None],
                    [256, 3, None],
                    [256, 3, None],
                    [256, 3, 3]
                    ]

fully_layers = [1024, 1024]
l0 = 1014
alphabet_size = 69
nb_classes = 4
batch_size = 32

model = Char_CNN(conv_layers=conv_layers,
                 fully_layers=fully_layers,
                 l0=l0,
                 alphabet_size=alphabet_size,
                 nb_classes=nb_classes,
                 batch_size=batch_size
                )
model.init()

In [66]:
import numpy as np
x = torch.randn(batch_size, alphabet_size, l0)
x = Variable(x)

In [67]:
model(x)


torch.Size([32, 69, 1014])
torch.Size([32, 256, 336])
torch.Size([32, 256, 110])
torch.Size([32, 256, 108])
torch.Size([32, 256, 106])
torch.Size([32, 256, 104])
torch.Size([32, 256, 34])
torch.Size([32, 8704])
torch.Size([32, 1024])
torch.Size([32, 1024])


Variable containing:
 0.2459  0.2484  0.2456  0.2601
 0.2499  0.2448  0.2483  0.2570
 0.2452  0.2495  0.2478  0.2575
 0.2485  0.2475  0.2466  0.2574
 0.2478  0.2481  0.2503  0.2538
 0.2482  0.2498  0.2430  0.2590
 0.2463  0.2471  0.2496  0.2571
 0.2490  0.2499  0.2467  0.2544
 0.2473  0.2462  0.2481  0.2585
 0.2435  0.2513  0.2458  0.2594
 0.2456  0.2499  0.2469  0.2576
 0.2460  0.2482  0.2492  0.2566
 0.2455  0.2492  0.2487  0.2567
 0.2447  0.2509  0.2473  0.2572
 0.2496  0.2464  0.2444  0.2596
 0.2474  0.2480  0.2477  0.2570
 0.2451  0.2481  0.2496  0.2573
 0.2457  0.2478  0.2496  0.2569
 0.2486  0.2486  0.2463  0.2565
 0.2455  0.2511  0.2458  0.2576
 0.2459  0.2467  0.2491  0.2582
 0.2470  0.2476  0.2482  0.2571
 0.2474  0.2513  0.2430  0.2583
 0.2449  0.2482  0.2499  0.2569
 0.2457  0.2493  0.2471  0.2579
 0.2476  0.2462  0.2493  0.2569
 0.2465  0.2457  0.2500  0.2578
 0.2486  0.2454  0.2512  0.2547
 0.2442  0.2523  0.2486  0.2548
 0.2476  0.2482  0.2484  0.2558
 0.2482  0.2509  0.

In [57]:
34*256

8704

In [52]:
102*1024

104448